In [1]:
import pandas as pd


In [2]:
#importamos el dataset
dataset = pd.read_excel("./src/lenses.ods")

#seleccionamos la condición de ejemplo positivo
target = dataset['target']
corpus = dataset.drop(['target'], axis=1).drop(['#'], axis=1)

print(dataset)
print(corpus)
print(target)

     #  Age  Spectacle_prescription  Astigmatic  Tear_production_rate  target
0    1    1                       1           1                     1       3
1    2    1                       1           1                     2       2
2    3    1                       1           2                     1       3
3    4    1                       1           2                     2       1
4    5    1                       2           1                     1       3
5    6    1                       2           1                     2       2
6    7    1                       2           2                     1       3
7    8    1                       2           2                     2       1
8    9    2                       1           1                     1       3
9   10    2                       1           1                     2       2
10  11    2                       1           2                     1       3
11  12    2                       1           2                 

In [3]:
#probemos que podemos usar la librería que creamos

from procesamiento_datasets import valores_observados, repeticiones_de_valor

In [4]:
#generamos las hipotesis general y especifica

from hipotesis import General, Especifica

import hipotesis


#recordemos que no hablamos de hipotesis general y especifica como unicas hipoesis, sino mas bien nos referimos a un conjunto de hipotesis
hipotesis_generales = [General(corpus.columns.tolist())]
hipotesis_especificas = [Especifica(corpus.columns.tolist())]

print(corpus.columns)

print(hipotesis_generales[0])
print(hipotesis_especificas[0])

Index(['Age', 'Spectacle_prescription', 'Astigmatic', 'Tear_production_rate'], dtype='object')
<*,*,*,*>
<0,0,0,0>


In [5]:
#como nuestro target son 3 clases, escojamos que es lo que queremos detectar (cuales son los casos positivos)

casos_positivos = [False, False, False] #<- el index-1 de esos booleanos indica que clase consideramos positiva

#Para efectos de esta muestra, vamos a hacer que los casos positivos seran el los primeros y los segundos, que representan a los que si requieren lentes de contacto

casos_positivos[0] = True
casos_positivos[1] = True

In [6]:
#comenzamos a recorrer el dataset para ir moldeando nuestras hipotesis

print("es esta impresion: ",hipotesis_especificas[0].etiquetas)
print("es esta impresion: ",hipotesis_especificas[0].aceptados)

for row, y in zip(corpus.values.tolist(), target):
    print(row, y)

    if casos_positivos[y-1]:
        #se trata de un ejemplo positivo, hay que generalizar nuestras hipotesis
        
        #primero comparamos cada elemento de nuestra hipotesis especifica

        for hipotesis_especifica in hipotesis_especificas:

            for aceptados_atributo, valor_observado, etiqueta in zip(hipotesis_especifica.aceptados, row, corpus.columns):

                print(etiqueta, valor_observado)

                if not aceptados_atributo.__contains__(valor_observado):
                    #aqui es donde hay que generalizar la hipotesis especifica

                    hipotesis_especifica.generalizar(etiqueta, valor_observado)

            print(hipotesis_especifica)
            print(hipotesis_especifica.aceptados)

        # break
            
        #ahora en la hipotesis general, tenemos que eliminar cualquier hipotesis que no cubra al caso positivo

        #si la hipotesis general es universal, tenemos que sustituir por las minimas especializaciones

        if hipotesis_generales[0].acepta_todo:
            
            #construimos el dominio de valores observados por cada atributo
            dominios:list[list[str]] = []

            for column in corpus.columns.tolist():
                
                dominios.append(valores_observados(corpus[column]))

            #sustituimos por minimas especializaciones
            hipotesis_generales = hipotesis.minimas_Especializaciones(hipotesis_generales[0], dominios)

            #eliminamos hipotesis que no cubren el caso positivo
            hipotesis_generales = hipotesis.validar_hipotesis_generales(hipotesis_generales, row)

        else:
            
            #Si las hipotesis generales ya no aceptan a todos los eleme4ntos, de las hipotesis generales que tenemos, solamente eliminamos las que no cubren al caso positivo
            hipotesis_generales = hipotesis.validar_hipotesis_generales(hipotesis_generales, row)

    else:

        #se trata de un ejemplo negatigo, hay que especificar la hipotesis

        #si la hipotesis general es universal, tenemos que sustituir por las minimas especializaciones

        if hipotesis_generales[0].acepta_todo:
            
            #construimos el dominio de valores observados por cada atributo
            dominios:list[list[str]] = []

            for column in corpus.columns.tolist():
                
                dominios.append(valores_observados(corpus[column]))

            print(dominios)

            #sustituimos por minimas especializaciones
            hipotesis_generales = hipotesis.minimas_Especializaciones(hipotesis_generales[0], dominios)

        #revisar que la hipotesis general no contenga al caso negativo o bien, que el caso negativo esté guardado en la lista de valores rechazados

        # for hipotesis_general in hipotesis_generales:

        #     for rechazados_atributo, valor_observado, etiqueta in zip(hipotesis_general.rechazados, row, corpus.columns):

        #         if not rechazados_atributo.__contains__(valor_observado):

        #             hipotesis_general.especializar(etiqueta, valor_observado)

        hipotesis_generales = hipotesis.validar_hipotesis_generales(hipotesis_generales, row, caso_positivo=False)

        # print("hola")

    print("evolucion de hipotesis: \n\n")

    for hipotesis_general in hipotesis_generales:

        print("general: \t",hipotesis_general)

    for hipotesis_especifica in hipotesis_especificas:
        
        print("especifica: \t", hipotesis_especifica)

    print(len(hipotesis_generales))


es esta impresion:  ['Age', 'Spectacle_prescription', 'Astigmatic', 'Tear_production_rate']
es esta impresion:  [[], [], [], []]
[1, 1, 1, 1] 3
[[1, 2, 3], [1, 2], [1, 2], [1, 2]]
hipotesis que estamos evaluando: <[1], *, *, *>

valor observado: 1	 lista de aceptados: [1]
valor observado: 1	 lista de aceptados: []
valor observado: 1	 lista de aceptados: []
valor observado: 1	 lista de aceptados: []
no mochamos nada
hipotesis que estamos evaluando: <[2], *, *, *>

valor observado: 1	 lista de aceptados: [2]
valor observado: 1	 lista de aceptados: []
valor observado: 1	 lista de aceptados: []
valor observado: 1	 lista de aceptados: []
no mochamos nada
hipotesis que estamos evaluando: <[3], *, *, *>

valor observado: 1	 lista de aceptados: [3]
valor observado: 1	 lista de aceptados: []
valor observado: 1	 lista de aceptados: []
valor observado: 1	 lista de aceptados: []
no mochamos nada
hipotesis que estamos evaluando: <*, [1], *, *>

valor observado: 1	 lista de aceptados: []
valor obser